# PDG API tutorial, PyHEP 2024

## Introduction

The goal of this tutorial is to provide a set of examples that illustrate the main features of the API. Basic familiarity with Python is assumed. Comprehensive documentation is available at https://pdgapi.lbl.gov/doc/

## Installation

First, create a virtual environment and activate it:

```bash
python -m venv ~/pdg.venv
source ~/pdg.venv/bin/activate
```

This ensures that any dependencies will be installed in a self-contained environment.

Now install the package:

```bash
pip install pdg
```

In order to run the examples in this notebook, some additional packages are required:

```bash
pip install numpy matplotlib jupyter
```

You can now launch a Jupyter session (`jupyter lab`) and open this notebook.

## Preamble

The examples that follow all assume that the following preamble has been run:

In [ ]:
import pdg
import matplotlib.pyplot as plt
import numpy as np

api = pdg.connect()

## An appetizer

Before we run through the fundamentals, let's take a look at a practical example that demonstrates one of the main new things offered by the API, namely, programmatic access to branching fractions:

In [ ]:
for bf in api.get_particle_by_name('K+').exclusive_branching_fractions():
    if not bf.is_limit:
        print(f'{bf.description:35} {bf.value:7g}')

This should be pretty self explanatory: We're just printing the exclusive branching fractions of the $K^+$. We'll show some other practical examples soon, but first, let's quickly go over the basics.

## The basics

### Getting a particle

Particles are represented by the `PdgParticle` class. There are multiple ways to get a particle from the API. Depending on the method, the result can be a `PdgParticleList`, a Python list of `PdgParticle`s, or a specific `PdgParticle`.

#### By name

A particle name can refer to a single particle or to a group of them. The function `get_particle_by_name` will return a `PdgParticle` if there's a unique match, and will raise an exception if not. The function `get_particles_by_name` always returns a list of `PdgParticles`. Aliases will automatically be resolved.

In [ ]:
api.get_particle_by_name('pi+')

In [ ]:
api.get_particles_by_name('pi')

#### By MC ID

Since MC IDs are unambiguous, `get_particle_by_mcid` returns a `PdgParticle` directly.

In [ ]:
api.get_particle_by_mcid(2212)

#### By PDG identifier

PDG identifiers correspond to pages on pdgLive and are shown on the page and URL. An identifier can refer to, e.g., a particle, or a set of measurements (e.g. masses) for a particle, or more general measurements, such as mixing angles. The `get` function, given a particle's PDG identifier, returns a list of all `PdgParticle`s associated to the identifier. Since `get` must return an instance of a `PdgData` subclass, the return type is a `PdgParticleList`, rather than a simple list of `PdgParticle`s.

In [ ]:
plist = api.get('S008/2024')
plist

However, a simple list is simple to get:

In [ ]:
list(plist)

### Getting quantum numbers

Quantum numbers are associated directly with a `PdgParticle` and can always be accessed as attributes.

In [ ]:
p = api.get_particle_by_name('pi0')
p.charge, p.quantum_I, p.quantum_G, p.quantum_J, p.quantum_P, p.quantum_C

In [ ]:
p = api.get_particle_by_name('pi+')
p.charge, p.quantum_I, p.quantum_G, p.quantum_J, p.quantum_P, p.quantum_C

### Getting masses, widths, and lifetimes

Masses, widths, and lifetimes are the main particle properties provided by the API. In some cases there may be multiple identifiers for masses (or widths or lifetimes) for a given particle, corresponding to different techniques or assumptions. A given particle may have width or lifetime properties, but not both.

The `masses`, `widths`, and `lifetimes` methods provide iterators over these properties.

In [ ]:
p = api.get_particle_by_name('pi+')
list(p.masses())

In [ ]:
list(p.widths())

In [ ]:
list(p.lifetimes())

The `PdgMass`, `PdgWidth`, and `PdgLifetime` classes are all subclasses of `PdgProperty`, which provides a `summary_values` function that produces a list of `SummaryTableValue` objects:

In [ ]:
api.default_edition

In [ ]:
m = api.get('S008M')
m

In [ ]:
m.summary_values()

Human-readable information on summary values is available via attributes:

In [ ]:
sv = m.summary_values()
sv[0].value_type, sv[1].value_type

In this case, we see that two summary values are provided for `S008M`, corresponding to the PDG fit and average.

A given summary value is encoded in whatever units are considered most appropriate. The encoded value and units can be accessed via the `value` and `units` attributes. The `get_value` method can be used to get the value in specified units.

In [ ]:
v = m.summary_values()[0]

v.value, v.units, v.get_value('GeV')

The `error`, `error_positive`, and `error_negative` attributes, and the `get_error` method, are analogous:

In [ ]:
v.error, v.error_positive, v.error_negative, v.get_error('GeV')

For convenience, the `mass`, `width`, and `lifetime` attributes of a `PdgParticle` can be used to get the "best" (i.e. first) summary value, in standard units of GeV and seconds. Normally, widths and lifetimes are automatically interconverted when necessary. (In pedantic mode, this does not occur, and there must be exactly one matching property identifier and summary value, or a `PdgNoDataError` will be thrown.)

In [ ]:
p.mass, p.width, p.lifetime

The top quark provides an example where there are multiple mass properties:

In [ ]:
p = api.get_particle_by_name('t')

list(p.masses())

In [ ]:
[m.description for m in p.masses()]

### Getting decays

In addition to masses and widths/lifetimes, a particle can have one or more branching fraction properties (which are either inclusive or exclusive):

In [ ]:
p = api.get_particle_by_name('pi+')

We can easily get all of the branching fractions of the $\pi^+$:

In [ ]:
list(p.branching_fractions())

In this case, they're all exclusive:

In [ ]:
list(p.inclusive_branching_fractions())

Taking the first decay:

In [ ]:
decay = next(p.exclusive_branching_fractions())
decay

We can get its description:

In [ ]:
decay.description

And the summary values for the branching fraction itself:

In [ ]:
decay.summary_values()

Going further, we can inspect the products of the decay:

In [ ]:
products = decay.decay_products
products

In [ ]:
products[0].item, products[0].multiplier, products[0].subdecay

## Some examples

Having covered the basics, let's dive into some more interesting examples.

### Printing all $B^0$ decays that produce a $J/\psi$

In [ ]:
p = api.get_particle_by_name('B0')
# The "canonical" name of the particle should be used for comparisons:
jpsi_name = api.get_canonical_name('J/psi')
# For the J/psi the canonical name is J/psi(1S)

for decay in p.exclusive_branching_fractions():
    for decay_product in decay.decay_products:
        item = decay_product.item
        # A decay product's PdgItem may be associated with a specific particle,
        # but may also be something more generic (e.g. "leptons"). If has_particle
        # is True, we can retrieve the associated PdgParticle via the particle property.
        if item.has_particle and item.particle.name == jpsi_name:
            print(f'{decay.description:65s} {decay.display_value_text}')

### Plotting masses of all decay products of the $D^+$

In [ ]:
all_masses = set()
p = api.get_particle_by_name('D+')
for decay in p.exclusive_branching_fractions():
    for prod in decay.decay_products:
        if not prod.item.has_particle:
            continue
        if prod.item.particle.has_mass_entry:
            # Since a mass entry could be a limit, rather than a measurement,
            # we must perform a check:
            if prod.item.particle.mass is not None:
                all_masses.add(prod.item.particle.mass)
plt.hist(all_masses);
plt.xlabel('Mass [GeV]')
plt.title('Masses of $D^+$ decay products');

### Plotting $K^+$ mass over time

For this, you will need the "pdgall" database file from https://pdg.lbl.gov/2024/api/index.html.

In [ ]:
api_all = pdg.connect("sqlite:///pdgall-2024-v0.1.0.sqlite")

In [ ]:
xs, ys, yerrs = [], [], []
for edition in api_all.editions:
    p = api_all.get_particle_by_name('K+', edition=edition)
    if p.has_mass_entry and p.mass is not None:
        xs.append(int(edition))
        ys.append(p.mass)
        yerrs.append(p.mass_error)

plt.errorbar(xs, ys, yerrs)
plt.xlabel('PDG edition')
plt.ylabel('Mass [GeV]')
plt.title('$K^+$ mass over time');

### Printing all neutrino mixing properties

Neutrino mixing properties all live under the `S067` parent identifier (as can be seen, for example, by browsing pdgLive). Let's print them all:

In [ ]:
parent = api.get('S067')
for prop in parent.get_children():
    if prop.has_best_summary():
        print(f'{prop.baseid:10s} {prop.value:8f} {prop.description}')

## Wrapping up

### Pedantic mode

Before we close, there's one important feature of the API that bears mentioning: *pedantic mode*. In pedantic mode, the API will not automatically choose the "best" property or summary value when multiple options are available. Instead, an exception will be thrown in such situations.

For example, we saw earlier that the top quark has three mass properties. Outside of pedantic mode, the API will choose the "best" (first) option:

In [ ]:
top = api.get_particle_by_name('t')
list(top.masses())

In [ ]:
top.mass

In pedantic mode, this is not the case:

In [ ]:
api_pedantic = pdg.connect(pedantic=True)

try:
    api_pedantic.get_particle_by_name('t').mass
except Exception as e:
    print(type(e), e)

In such a situation, pedantic mode will require youto be explicit about which mass you want:

In [ ]:
api_pedantic.get('Q007TP').best_summary().get_value('GeV')

### Closing words

This concludes our tour of the PDG API. Thanks for following along! We hope that you are now positioned to start making use of the API, and we welcome your feedback.